# 한글폰트 설치
- 아래 실행후 '런타임 다시 시작' 하면 됩니다.
- 메뉴에서 '런타임' >> '런타임 다시 시작'을 클릭하세요.
- 런타임 다시 시작 후 다시 실행할 필요 없습니다.

In [1]:
# 폰트 설치
!apt-get update -qq # 나눔고딕 설치
!apt-get install fonts-nanum* -qq

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 로딩
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

Selecting previously unselected package fonts-nanum.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


# Install

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 4.5MB/s 


# Evn

In [2]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [3]:
from sklearn.decomposition import PCA

In [4]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [5]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Gensim
- https://radimrehurek.com/gensim/index.html

In [6]:
import gensim
import gensim.downloader as api

## 한국어 학습

- https://wikidocs.net/50739

In [ ]:
# 행태소분석기 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [8]:
import konlpy
okt = konlpy.tag.Okt()

In [9]:
okt.morphs("아버지가방에들어가신다")

['아버지', '가방', '에', '들어가신다']

In [10]:
# 네이버 영화 리뷰 데이터 다운로드
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

--2021-01-27 14:52:57--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  39.5MB/s    in 0.5s    

2021-01-27 14:52:58 (39.5 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



In [11]:
nsmc_data = pd.read_csv("ratings.txt", header=0, delimiter='\t', quoting=3)
print(f"전체 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

전체 데이터의 개수: 200000


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [12]:
# null 제거
nsmc_data = nsmc_data.dropna()
print(f"null 제거 후 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

null 제거 후 데이터의 개수: 199992


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [13]:
# 한글 이외의 문자 제거
nsmc_data['document'] = nsmc_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
nsmc_data.dropna()
print(f"한글 아닌 문자 제거 후 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

한글 아닌 문자 제거 후 데이터의 개수: 199992


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁 나오나요 나오면 빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [14]:
# 불용어 정의 (빈도가 너무 많은 단어는 학습에서 제외 함)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [15]:
# okt 형태소 분석기를 이용해 형태소 단위로 분할 (이때 불용어 제거)
tokens = []
for i, document in enumerate(tqdm(nsmc_data['document'], total=len(nsmc_data))):
    line = []
    line = okt.morphs(document)
    line = [word for word in line if not word in stopwords]
    tokens.append(line)

In [16]:
print(len(tokens))
tokens[:10]

199992


[['어릴', '때', '보고', '지금', '다시', '봐도', '재밌어요', 'ㅋㅋ'],
 ['디자인',
  '을',
  '배우는',
  '학생',
  '외국',
  '디자이너',
  '그',
  '일군',
  '전통',
  '을',
  '통해',
  '발전',
  '해가는',
  '문화',
  '산업',
  '부러웠는데',
  '사실',
  '우리나라',
  '에서도',
  '그',
  '어려운',
  '시절',
  '끝',
  '까지',
  '열정',
  '을',
  '지킨',
  '노라노',
  '같은',
  '전통',
  '있어',
  '저',
  '같은',
  '사람',
  '꿈',
  '을',
  '꾸고',
  '이뤄',
  '나갈',
  '수',
  '있다는',
  '것',
  '감사합니다'],
 ['폴리스스토리', '시리즈', '부터', '뉴', '까지', '버릴께', '하나', '없음', '최고'],
 ['연기',
  '진짜',
  '개',
  '쩔구나',
  '지루할거라고',
  '생각',
  '했는데',
  '몰입',
  '해서',
  '봤다',
  '그래',
  '이런게',
  '진짜',
  '영화',
  '지'],
 ['안개', '자욱한', '밤하늘', '떠', '있는', '초승달', '같은', '영화'],
 ['사랑', '을', '해본', '사람', '라면', '처음', '부터', '끝', '까지', '웃을수', '있는', '영화'],
 ['완전', '감동', '입니다', '다시', '봐도', '감동'],
 ['개', '전쟁', '나오나요', '나오면', '빠', '로', '보고', '싶음'],
 ['굿'],
 ['바보', '아니라', '병', '쉰', '인듯']]

In [17]:
# gensim 학습
word2vec_100 = gensim.models.Word2Vec(sentences=tokens, size=100, window=5, min_count=5)

In [18]:
words = list(word2vec_100.wv.vocab)
len(words), words[:100]

(26174,
 ['어릴',
  '때',
  '보고',
  '지금',
  '다시',
  '봐도',
  '재밌어요',
  'ㅋㅋ',
  '디자인',
  '을',
  '배우는',
  '학생',
  '외국',
  '디자이너',
  '그',
  '전통',
  '통해',
  '발전',
  '해가는',
  '문화',
  '산업',
  '사실',
  '우리나라',
  '에서도',
  '어려운',
  '시절',
  '끝',
  '까지',
  '열정',
  '지킨',
  '같은',
  '있어',
  '저',
  '사람',
  '꿈',
  '꾸고',
  '이뤄',
  '나갈',
  '수',
  '있다는',
  '것',
  '감사합니다',
  '폴리스스토리',
  '시리즈',
  '부터',
  '뉴',
  '하나',
  '없음',
  '최고',
  '연기',
  '진짜',
  '개',
  '생각',
  '했는데',
  '몰입',
  '해서',
  '봤다',
  '그래',
  '이런게',
  '영화',
  '지',
  '안개',
  '밤하늘',
  '떠',
  '있는',
  '사랑',
  '해본',
  '라면',
  '처음',
  '웃을수',
  '완전',
  '감동',
  '입니다',
  '전쟁',
  '나오나요',
  '나오면',
  '빠',
  '로',
  '싶음',
  '굿',
  '바보',
  '아니라',
  '병',
  '쉰',
  '인듯',
  '내',
  '나이',
  '본',
  '나',
  '적',
  '이다',
  '하지만',
  '훗날',
  '사하나',
  '하',
  '감정',
  '완벽하게',
  '이해',
  '같다',
  '재밌다'])

In [19]:
similar = word2vec_100.wv.most_similar("영화")
similar

[('애니', 0.6344777345657349),
 ('공포영화', 0.6149888038635254),
 ('애니메이션', 0.6083066463470459),
 ('독립영화', 0.6001555919647217),
 ('다큐', 0.5794230699539185),
 ('액션영화', 0.5489228963851929),
 ('작품', 0.5444856882095337),
 ('다큐멘터리', 0.5375467538833618),
 ('재난영화', 0.533909797668457),
 ('공포물', 0.5216571092605591)]

In [20]:
similar = word2vec_100.wv.most_similar("최민수")
similar

[('안성기', 0.930884063243866),
 ('한석규', 0.9102240204811096),
 ('김승우', 0.9078730940818787),
 ('크로우', 0.9063593149185181),
 ('박신양', 0.906130313873291),
 ('엄태웅', 0.9061157703399658),
 ('러셀', 0.9017555713653564),
 ('장미희', 0.9010257124900818),
 ('이정재', 0.9009615182876587),
 ('박용우', 0.8979752063751221)]

In [21]:
similar = word2vec_100.wv.most_similar("장동건")
similar

[('심은경', 0.8964840173721313),
 ('난쟁이', 0.8846665620803833),
 ('앨', 0.8841480016708374),
 ('조한선', 0.8818334937095642),
 ('양자경', 0.8810635209083557),
 ('박신양', 0.8786536455154419),
 ('이미숙', 0.8773421049118042),
 ('홉킨스', 0.8761685490608215),
 ('안성기', 0.8759670257568359),
 ('공리', 0.8751939535140991)]

In [22]:
# 설경구 - 송윤아 + 고소영
result = word2vec_100.wv.most_similar(positive=['고소영', '설경구'], negative=['송윤아'])
result

[('송강호', 0.8507436513900757),
 ('혼신', 0.8417149782180786),
 ('차승원', 0.8221091628074646),
 ('정우성', 0.8217061161994934),
 ('김혜수', 0.8121574521064758),
 ('박중훈', 0.8046863079071045),
 ('임창정', 0.8042894601821899),
 ('한석규', 0.7919398546218872),
 ('디카프리오', 0.7840991020202637),
 ('러셀', 0.7838752269744873)]